---
## Module Import
---

In [1]:
import pandas as pd
import os
import gensim
import numpy as np
import time
import pickle
import warnings
from datetime import datetime
from datetime import date, timedelta
from imblearn.over_sampling import RandomOverSampler
from gensim.models.fasttext import FastText as FT_gensim
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Conv1D, GlobalMaxPool1D, SimpleRNN
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
import pymysql

from DNS_Encoding.data_nodns_del import data_nodns_del
from DNS_Encoding.make_fasttext_model_vector2 import make_fasttext_model_vector2
from DNS_Encoding.making_training_bydns import making_training_bydns
from IP_Encoding.ipencoding_65536_delH import ipencoding_65536_delH
from IP_Encoding.ipencoding_65536_delH import parallelize_dataframe
from Etc_features.age_onehot import age_onehot
from Etc_features.gender_onehot import gender_onehot
from Etc_features.port_onehot import port_onehot
from Etc_features.protocol_onehot import protocol_onehot
from Etc_features.time_hour_onehot import time_hour_onehot
from Etc_features.train_delta_dn_link_data_size import train_delta_dn_link_data_size
from Etc_features.train_delta_up_link_data_size import train_delta_up_link_data_size
from Etc_features.train_lastdnstimetolive import train_lastdnstimetolive
from Etc_features.train_synackmsssize import train_synackmsssize
from Etc_features.train_delta_dn_link_payload_size import train_delta_dn_link_payload_size
from Etc_features.train_delta_up_link_payload_size import train_delta_up_link_payload_size


Using TensorFlow backend.


---
## HTTP_KNOWN : 권 대리님이 나누신 것
---

In [2]:
current = '2020-01-21 01:00'
before1hour = '2020-01-21 00:00'
datatable = '20200121'
before1hour_date = 'T_AD_XDR_HTTP_KNOWN_' + datatable

conn = pymysql.connect(host ='192.168.6.89' , port = 3306,
                       user = 'atom', password = 'atom', db = '5G_Probe_App_Discovery', charset = 'utf8',
                      cursorclass = pymysql.cursors.DictCursor)

# 커서 가져오기 - 시간 정확히 다시 확인 필요
curs = conn.cursor()
sql = "SELECT * FROM " + before1hour_date + " WHERE OngoingFlag = 1 AND SummaryCreateTime >= unix_timestamp(" +"'"+before1hour+"'"+") AND SummaryCreateTime < unix_timestamp(" +"'"+ current +"');"
curs.execute(sql)

# 데이터 가져오기
http = pd.DataFrame(curs.fetchall())
http = http[['SummaryCreateTime', 'IMSI', 'Gender', 'Age', 'App_ID', 'App_Group_Code', 'App_Port','Protocol', 'IP_Version', 'App_IPv4', 'App_IPv6',  'DNS.QueryName', 'HTTP.Request.Host', 'DNS.LastDnsTimeToLive', 'Traffic.Delta.Dnlink.DataSize', 'Traffic.Delta.Uplink.DataSize', 'TCP.Delta.MSS.SynMssSize', 'TCP.Delta.MSS.SynAckMssSize', 'Traffic.Delta.Uplink.RetransPayloadSize', 'Traffic.Delta.Dnlink.RetransPayloadSize', 'HTTP.Delta.Quility.TotResponseTime']].copy()

print('http_appid 개수:' + str(len(list(http.groupby('App_ID').count().index))))

"""import pandas as pd
http = pd.read_csv('http_classifier_train_20191212.csv')

http = http[['summary_create_time', 'imsi', 'gender', 'age', 'app_id', 'app_group_cd', 'protocol', 'app_ip_version', 'app_ip_v4', 'app_ip_v6', 'app_port', 'dns_querey_name', 'request_host', 'lastdnstimetolive', 'delta_dn_link_data_size', 'delta_up_link_data_size', 'synackmsssize', 'delta_dn_link_payload_size','delta_up_link_payload_size', 'quality_total_response_time', 'synmsssize']]
http[['summary_create_time']] = http[['summary_create_time']].apply(lambda x: x.astype(float))
http[['protocol', 'app_ip_version', 'app_ip_v4', 'app_port', 'lastdnstimetolive', 'delta_dn_link_data_size', 'delta_up_link_data_size', 'synackmsssize']] = http[['protocol', 'app_ip_version', 'app_ip_v4', 'app_port', 'lastdnstimetolive', 'delta_dn_link_data_size', 'delta_up_link_data_size', 'synackmsssize']].apply(lambda x: x.astype(int))
http[['gender','age','app_id','app_group_cd', 'app_ip_v6', 'dns_querey_name', 'request_host']] = http[['gender','age','app_id','app_group_cd', 'app_ip_v6', 'dns_querey_name', 'request_host']].apply(lambda x: x.astype(object))"""

http_appid 개수:257


"import pandas as pd\nhttp = pd.read_csv('http_classifier_train_20191212.csv')\n\nhttp = http[['summary_create_time', 'imsi', 'gender', 'age', 'app_id', 'app_group_cd', 'protocol', 'app_ip_version', 'app_ip_v4', 'app_ip_v6', 'app_port', 'dns_querey_name', 'request_host', 'lastdnstimetolive', 'delta_dn_link_data_size', 'delta_up_link_data_size', 'synackmsssize', 'delta_dn_link_payload_size','delta_up_link_payload_size', 'quality_total_response_time', 'synmsssize']]\nhttp[['summary_create_time']] = http[['summary_create_time']].apply(lambda x: x.astype(float))\nhttp[['protocol', 'app_ip_version', 'app_ip_v4', 'app_port', 'lastdnstimetolive', 'delta_dn_link_data_size', 'delta_up_link_data_size', 'synackmsssize']] = http[['protocol', 'app_ip_version', 'app_ip_v4', 'app_port', 'lastdnstimetolive', 'delta_dn_link_data_size', 'delta_up_link_data_size', 'synackmsssize']].apply(lambda x: x.astype(int))\nhttp[['gender','age','app_id','app_group_cd', 'app_ip_v6', 'dns_querey_name', 'request_host'

---
#### 데이터 만들때는 app_id 개수가 5개이하인건 제외하고 생각
#### 이상치 알고리즘에서 6개이상인것만 신경쓰도록 해야한다. 5개이하인 app_id 개수 : 156개
#### 6개이상인 appid에 대해서 2개씩을 이상치로 놓음
---

In [3]:
http['new_label'] = http['App_ID']
http.head()

,SummaryCreateTime,IMSI,Gender,Age,App_ID,App_Group_Code,App_Port,Protocol,IP_Version,App_IPv4,...,HTTP.Request.Host,DNS.LastDnsTimeToLive,Traffic.Delta.Dnlink.DataSize,Traffic.Delta.Uplink.DataSize,TCP.Delta.MSS.SynMssSize,TCP.Delta.MSS.SynAckMssSize,Traffic.Delta.Uplink.RetransPayloadSize,Traffic.Delta.Dnlink.RetransPayloadSize,HTTP.Delta.Quility.TotResponseTime,new_label
0,1.579532e+09,T0kFaqCe3jNqG3pf1e67TfRA==,1,25,ED08,SNS,8080,6,2,0,...,,0,580,1049,0,0,0,0,0,ED08
1,1.579532e+09,T0cFTF+m9S5YO8h7dfMivTmQ==,1,27,C350,Multimedia,80,6,2,3031841756,...,cdnimg.melon.co.kr,0,492,531,0,0,0,0,4888,C350
2,1.579532e+09,T0cFTF+m9S5YO8h7dfMivTmQ==,1,27,C350,Multimedia,80,6,2,3740226946,...,smmd.melon.com,0,586,750,0,0,0,0,14299,C350
3,1.579532e+09,T0cFTF+m9S5YO8h7dfMivTmQ==,1,27,C350,Multimedia,80,6,2,2033571853,...,log.melon.com,0,598,2497,0,0,0,0,6423,C350
4,1.579532e+09,T0HyOiSk74dwmP0QmPCWUjxg==,1,40,EBF3,Web,80,6,2,3419275251,...,mmsns.qpic.cn,0,5522,1543,0,0,0,0,41800,EBF3


In [4]:
http.groupby('App_ID').size().sort_values()[230:] #550

App_ID
EAA0     106
C751     121
C369     134
C356     150
EF07     153
C672     157
EBAD     163
EBA0     168
C35E     190
C5C1     197
F048     206
C5C4     214
C56F     221
C3B8     223
EDF8     249
C478     251
C351     260
C3E8     266
C5C5     537
C352     543
C36E     588
C4CD     903
C7DD     918
C35B    1226
C350    1367
ED08    1598
EA99    1645
dtype: int64

In [5]:
appid_6 = list(http.groupby('App_ID').size().sort_values()[230: ].index)
appid_6[:5]

['EAA0', 'C751', 'C369', 'C356', 'EF07']

In [6]:
for i in range(len(appid_6)):
    if i < len(appid_6)-1:
        http.loc[list(http[http.App_ID == appid_6[i]].iloc[-100 : ].index)[:], 'new_label'] = appid_6[i+1]
        http.loc[list(http[http.App_ID == appid_6[i]].iloc[ : -100].index)[:], 'new_label'] = appid_6[i]
    else : 
        http.loc[list(http[http.App_ID == appid_6[i]].iloc[-100 : ].index)[:], 'new_label'] = appid_6[0] 
        http.loc[list(http[http.App_ID == appid_6[i]].iloc[ : -100].index)[:], 'new_label'] = appid_6[i]  

---
## DNS Vector화
---

In [7]:
http_1 = data_nodns_del(http)
http_vector, http_modelinit, max_length_http = make_fasttext_model_vector2(http_1)
httpdns_training = making_training_bydns(http, http_vector)

---
## IP Vector화
---

In [8]:
#http_ip65536_delH = ipencoding_65536_delH(http)
http_ip65536_delH = parallelize_dataframe(http, ipencoding_65536_delH)

---
## ETC Features Vecotr화 
---

In [9]:
http_port = port_onehot(http)
http_protocol = protocol_onehot(http)
http_lastdnstimetolive = train_lastdnstimetolive(http)
http_delta_dn_link_data_size = train_delta_dn_link_data_size(http)
http_delta_up_link_data_size = train_delta_up_link_data_size(http)
http_synackmsssize = train_synackmsssize(http)
http_train_delta_dn_link_payload_size = train_delta_dn_link_payload_size(http)
http_train_delta_up_link_payload_size = train_delta_up_link_payload_size(http)


http_training_ip65536_delH = pd.concat([http_ip65536_delH, httpdns_training,http_port, 
                                        http_protocol,http_lastdnstimetolive,http_delta_dn_link_data_size,http_delta_up_link_data_size,
                                        http_synackmsssize, http_train_delta_up_link_payload_size, http_train_delta_dn_link_payload_size], axis = 1)


In [10]:
app_id = list(http.groupby('App_ID').count().index)
http_training_ip65536_delH = pd.concat([http_training_ip65536_delH , http[['App_ID']]], axis=1)
http_training_ip65536_delH = pd.concat([http_training_ip65536_delH , http[['new_label']]], axis=1)
http_training_ip65536_delH.head()

,Aclass,Bclass,Cclass,Dclass,Eclass,Fclass,Gclass,0,1,2,...,protocol__6,protocol_x,lastdnstimetolive,delta_dn_link_data_size,delta_up_link_data_size,synackmsssize,delta_up_link_payload_size,delta_dn_link_payload_size,App_ID,new_label
0,0.140688,0.273442,0.000061,0.007996,0.007858,0.004685,0.005707,-0.054046,0.046158,-0.169848,...,1,0,0.0,9.676013e-07,0.000004,0.0,0.0,0.0,ED08,ED08
1,0.001526,0.998474,0.000000,0.000000,0.000000,0.000000,0.705913,-1.242447,-0.194013,-1.377864,...,1,0,0.0,8.207928e-07,0.000002,0.0,0.0,0.0,C350,C350
2,0.001526,0.998474,0.000000,0.000000,0.000000,0.000000,0.870848,-0.686407,-0.472209,-0.432687,...,1,0,0.0,9.776110e-07,0.000003,0.0,0.0,0.0,C350,C350
3,0.001526,0.998474,0.000000,0.000000,0.000000,0.000000,0.473472,-0.811053,-0.661471,-0.547387,...,1,0,0.0,9.976303e-07,0.000010,0.0,0.0,0.0,C350,C350
4,0.001526,0.998474,0.000000,0.000000,0.000000,0.000000,0.796109,-0.416056,0.044003,-0.564658,...,1,0,0.0,9.212232e-06,0.000006,0.0,0.0,0.0,EBF3,EBF3


---
#### isolation forest 1, -1
---

In [14]:
a = pd.DataFrame(columns = http_training_ip65536_delH.columns)


for i in range(len(app_id)):
    if len(http_training_ip65536_delH[http_training_ip65536_delH.App_ID == app_id[i]])<10:
        xx = http_training_ip65536_delH.loc[list(http_training_ip65536_delH[http_training_ip65536_delH.App_ID == app_id[i]].index)]
        xx['anomaly_score'] = [2 for i in range(len(xx))]
        a = pd.concat([a,xx])

        
###### 2는 3개이하짜리들 
    
    else:
        clf = IsolationForest(n_estimators=50, contamination=0.10, random_state=35, verbose=0)
        y_pred = clf.fit_predict(http_training_ip65536_delH[http_training_ip65536_delH.App_ID == app_id[i]].drop(['App_ID','new_label'], axis = 1)) #1,-1로 나온다.
        
        xx = http_training_ip65536_delH.loc[list(http_training_ip65536_delH[http_training_ip65536_delH.App_ID == app_id[i]].index)]
        xx['anomaly_score'] = list(y_pred)
        a = pd.concat([a,xx])
        
a        
###### -1이 이상치, 1이 정상.

/home/eunhye/anaconda3/envs/csb2/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/home/eunhye/anaconda3/envs/csb2/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
/home/eunhye/anaconda3/envs/csb2/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/eunhye/anaconda3/envs/csb2/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: be

,Aclass,Bclass,Cclass,Dclass,Eclass,Fclass,Gclass,0,1,2,...,protocol_x,lastdnstimetolive,delta_dn_link_data_size,delta_up_link_data_size,synackmsssize,delta_up_link_payload_size,delta_dn_link_payload_size,App_ID,new_label,anomaly_score
1,0.001526,0.998474,0.000000,0.000000,0.0,0.0,0.705913,-1.242447,-0.194013,-1.377864,...,0,0.0,8.207928e-07,2.117987e-06,0.0,0.0,0.0,C350,C350,1.0
2,0.001526,0.998474,0.000000,0.000000,0.0,0.0,0.870848,-0.686407,-0.472209,-0.432687,...,0,0.0,9.776110e-07,2.991507e-06,0.0,0.0,0.0,C350,C350,1.0
3,0.001526,0.998474,0.000000,0.000000,0.0,0.0,0.473472,-0.811053,-0.661471,-0.547387,...,0,0.0,9.976303e-07,9.959724e-06,0.0,0.0,0.0,C350,C350,1.0
10,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.473472,-0.811053,-0.661471,-0.547387,...,0,0.0,2.202127e-07,5.265052e-07,0.0,0.0,0.0,C350,C350,-1.0
33,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.705913,-1.242447,-0.194013,-1.377864,...,0,0.0,4.507838e-04,6.228716e-05,0.0,0.0,0.0,C350,C350,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15866,0.125017,0.011109,0.003418,0.019669,0.0,0.0,0.000000,-0.160806,0.010956,-0.235564,...,0,0.0,2.869438e-07,2.058157e-06,0.0,0.0,0.0,H014,H014,1.0
7350,0.140688,0.406256,0.250065,0.031617,0.0,0.0,0.000000,-0.287533,0.064278,-0.435250,...,0,0.0,1.372993e-06,2.066134e-06,0.0,0.0,0.0,H016,H016,2.0
7584,0.140688,0.406256,0.250065,0.031617,0.0,0.0,0.000000,-0.287533,0.064278,-0.435250,...,0,0.0,1.659937e-06,4.124291e-06,0.0,0.0,0.0,H016,H016,2.0
14673,0.001526,0.998474,0.000000,0.000000,0.0,0.0,0.204318,-1.895292,1.411006,-2.359557,...,0,0.0,2.293882e-06,3.725423e-06,0.0,0.0,0.0,IAAO,IAAO,2.0


In [16]:
x = sum(a[a.App_ID != a.new_label].anomaly_score == -1)
y = sum(a[a.App_ID != a.new_label].anomaly_score == 1)
print('이상치 중에서 실제로 이상치로 모델이 판별한 비율 : ' + str(x/(x+y)*100))

이상치 중에서 실제로 이상치로 모델이 판별한 비율 : 10.703703703703704


In [17]:
z = sum(a[a.App_ID == a.new_label].anomaly_score == -1)
w = sum(a[a.App_ID == a.new_label].anomaly_score == 1)
print('정상 중에서 이상치로 잘못 판별한 비율 : ' + str(z/(z+w)*100))

정상 중에서 이상치로 잘못 판별한 비율 : 9.998424204223133


---
#### isolation forest score 값
---

In [34]:
a = pd.DataFrame(columns = http_training_ip65536_delH.columns)

for i in range(len(app_id)):
    if len(http_training_ip65536_delH[http_training_ip65536_delH.app_id == app_id[i]])<1724:
        xx = http_training_ip65536_delH.loc[list(http_training_ip65536_delH[http_training_ip65536_delH.app_id == app_id[i]].index)]
        xx['anomaly_score'] = [2 for i in range(len(xx))]
        a = pd.concat([a,xx])

            
    else:
        clf = IsolationForest(n_estimators=50, contamination=0.01, random_state=35, verbose=0)
        clf.fit(http_training_ip65536_delH[http_training_ip65536_delH.app_id == app_id[i]].drop(['app_id','new_label'], axis = 1))
        y_pred = clf.score_samples(http_training_ip65536_delH[http_training_ip65536_delH.app_id == app_id[i]].drop(['app_id','new_label'], axis = 1)) #1,-1로 나온다.
        xx = http_training_ip65536_delH.loc[list(http_training_ip65536_delH[http_training_ip65536_delH.app_id == app_id[i]].index)]
        xx['anomaly_score'] = list(y_pred)
        a = pd.concat([a,xx])
        
        
###### -1이 이상치, 1이 정상.

/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()
/home/chkimsu/anaconda3/envs/csb2/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/h

In [35]:
print('전체 데이터 개수 : '+str(len(a)))
print('모델이 이상치로 판별한 데이터 개수 : ' + str(len(a[a.anomaly_score<-0.5])))
print('데이터 실제 이상치 개수 : ' + str(sum(a.app_id != a.new_label)))
print('이상치로 판별한 데이터 중 실제 이상치 개수 : '+ str(sum(a[a.anomaly_score<-0.5].app_id != a[a.anomaly_score<-0.5].new_label)))

전체 데이터 개수 : 215426
모델이 이상치로 판별한 데이터 개수 : 27189
데이터 실제 이상치 개수 : 22000
이상치로 판별한 데이터 중 실제 이상치 개수 : 3598


---
#### 권대리님 모델에 태울 것
---

In [37]:
http.loc[list(a[a.anomaly_score <-0.5].index)].to_csv('modelinputhttp.csv')